# Семантическая Сегментация. Часть 3.

## Переключение версии TensorFlow

In [1]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import os
import skimage.io as io
import numpy as np

import tensorflow as tf

"""
Пояснения к коду:

1. import os
   - Модуль os предоставляет функции для взаимодействия с операционной системой.
   - Используется для работы с файлами и папками, например, чтения путей, создания папок и т.п.
   - В задачах компьютерного зрения часто нужен для загрузки изображений из директорий.

2. import skimage.io as io
   - skimage (scikit-image) — библиотека для обработки изображений.
   - Модуль io отвечает за ввод-вывод изображений.
   - Функции этого модуля позволяют читать изображения из файлов и сохранять их.
   - Например, io.imread('path/to/image.jpg') — загрузит изображение в виде массива пикселей.

3. import numpy as np
   - numpy — основная библиотека для научных вычислений в Python.
   - Используется для работы с многомерными массивами данных, которые часто представляют изображения.
   - В компьютерном зрении изображения обычно хранятся как 2D (градации серого) или 3D (цветные) numpy-массивы.
   - np.array(...) создаёт или преобразует данные в массив numpy.
   - Предоставляет удобные операции над массивами: математические функции, изменение формы, срезы и т.д.

4. import tensorflow as tf
   - TensorFlow — мощная библиотека для машинного обучения и нейронных сетей от Google.
   - Используется для построения, обучения и применения моделей глубокого обучения.
   - tf содержит множество инструментов и API для создания сложных моделей, в том числе для компьютерного зрения.
   - TensorFlow работает с тензорами — многомерными массивами, похожими на numpy-массивы, но оптимизированными для вычислений на GPU.
   - Позволяет автоматизировать вычисление градиентов и обучение нейросетей.

Обобщение:
- Этот код импортирует базовые библиотеки, необходимые для чтения изображений (skimage.io), работы с массивами данных (numpy), управления файлами (os) и создания/использования моделей глубокого обучения (tensorflow).
- В компьютерном зрении именно такой набор — основа для загрузки данных и их обработки перед передачей в нейронную сеть.
"""


"\nПояснения к коду:\n\n1. import os\n   - Модуль os предоставляет функции для взаимодействия с операционной системой.\n   - Используется для работы с файлами и папками, например, чтения путей, создания папок и т.п.\n   - В задачах компьютерного зрения часто нужен для загрузки изображений из директорий.\n\n2. import skimage.io as io\n   - skimage (scikit-image) — библиотека для обработки изображений.\n   - Модуль io отвечает за ввод-вывод изображений.\n   - Функции этого модуля позволяют читать изображения из файлов и сохранять их.\n   - Например, io.imread('path/to/image.jpg') — загрузит изображение в виде массива пикселей.\n\n3. import numpy as np\n   - numpy — основная библиотека для научных вычислений в Python.\n   - Используется для работы с многомерными массивами данных, которые часто представляют изображения.\n   - В компьютерном зрении изображения обычно хранятся как 2D (градации серого) или 3D (цветные) numpy-массивы.\n   - np.array(...) создаёт или преобразует данные в массив

## Загрузка датасета COCO и COCO API

In [3]:
if 1:
    # Создаем папку 'data' (если она уже есть, ошибок не будет из-за -p)
    !mkdir -p data

    # Загрузка файла с аннотациями COCO для train и val (комментарии означают, что загрузка изображений train2017 и val2017 отключена)
    # !cd data && wget http://images.cocodataset.org/zips/train2017.zip
    # !cd data && wget http://images.cocodataset.org/zips/val2017.zip
    !cd data && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

    # Распаковка архива с аннотациями без вывода подробностей (-q — тихий режим)
    # Аналогично, распаковка train2017.zip и val2017.zip сейчас отключена
    # !cd data && unzip -q train2017.zip
    # !cd data && unzip -q val2017.zip
    !cd data && unzip -q annotations_trainval2017.zip

    # Клонирование репозитория COCO API, который нужен для удобной работы с датасетом COCO
    !cd data && git clone https://github.com/cocodataset/cocoapi

    # Переходим в папку с Python API для COCO и компилируем код с помощью команды make
    # Это подготовит необходимые Python-библиотеки для работы с COCO датасетом
    !cd data/cocoapi/PythonAPI && make


--2025-07-11 17:30:42--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.182.37.113, 16.15.184.203, 54.231.229.233, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.182.37.113|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  54.9MB/s    in 4.9s    

2025-07-11 17:30:47 (49.6 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975 (from 1)
Receiving objects: 100% (975/975), 11.72 MiB | 32.27 MiB/s, done.
Resolving deltas: 100% (576/576), done.
python setup.py build_ext --inplace
running build_ext
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx

## Подготовка COCO API

In [4]:
COCO_ROOT = './data/'  # Путь к папке с данными COCO (где лежит распакованный датасет и api)

import sys  # Импортируем модуль sys для работы с системными путями

# Добавляем в начало списка путей Python путь к папке с Python API COCO
# os.path.join объединяет пути корректно для любой ОС, создавая путь './data/cocoapi/PythonAPI'
sys.path.insert(0, os.path.join(COCO_ROOT, 'cocoapi/PythonAPI'))

# Импортируем класс COCO из модуля pycocotools.coco — это основной интерфейс для работы с аннотациями COCO
# COCO позволяет загружать аннотации, получать информацию об объектах, категориях, изображениях
from pycocotools.coco import COCO


## Универсальный класс Dataset для сегментации

In [5]:
class Dataset():

    def crop_images(self, img, inp_size, random_crop=False):
        # Получаем форму изображения (высота, ширина, количество каналов)
        shape = tf.shape(img)

        # Создаем паддинг (добавление пустых пикселей) по высоте и ширине,
        # чтобы довести изображение до размера inp_size, если оно меньше
        pad = (
            [0, tf.maximum(inp_size - shape[0], 0)],  # паддинг по высоте снизу
            [0, tf.maximum(inp_size - shape[1], 0)],  # паддинг по ширине справа
            [0, 0],                                  # без паддинга по каналам (цвету)
        )
        img = tf.pad(img, pad)  # добавляем паддинг к изображению

        if random_crop:
            # Если включен случайный кроп, вырезаем случайный квадрат размером inp_size x inp_size
            img = tf.image.random_crop(img, (inp_size, inp_size, shape[2]))
        else:
            # Иначе делаем центральный кроп — берем центральный квадрат inp_size x inp_size
            shape = tf.shape(img)
            ho = (shape[0] - inp_size) // 2  # смещение по высоте от центра
            wo = (shape[1] - inp_size) // 2  # смещение по ширине от центра
            img = img[ho:ho+inp_size, wo:wo+inp_size, :]  # вырезаем центр

        return img  # возвращаем обрезанное изображение

    def train_dataset(self, batch_size, epochs, inp_size):

        def item_to_images(item):
            random_crop = True  # при обучении используем случайный кроп для аугментации данных
            # Вызываем метод self.read_images (в другом месте класса), возвращающий изображение + маску
            # tf.py_function позволяет вызвать обычную Python-функцию внутри tf.data
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            # Обрезаем (кропим) изображение до нужного размера
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            # Отделяем первые 3 канала (RGB) и приводим к типу float32, нормируем в [0,1]
            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            # Отделяем маску — 4-й канал, преобразуем к float32
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class  # возвращаем пару (изображение, маска)

        # Создаем датасет из списка путей к изображениям (self.img_list)
        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        # Перемешиваем данные для обучения
        dataset = dataset.shuffle(buffer_size=len(self.img_list))
        # Применяем функцию обработки (item_to_images) ко всем элементам
        dataset = dataset.map(item_to_images)
        # Повторяем данные заданное число эпох
        dataset = dataset.repeat(epochs)
        # Формируем батчи заданного размера, отбрасываем остаток
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset  # возвращаем готовый датасет для обучения

    def val_dataset(self, batch_size, inp_size):

        def item_to_images(item):
            random_crop = False  # при валидации кроп делаем центральным для стабильности
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.map(item_to_images)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset  # возвращаем готовый датасет для валидации


## Класс для сегментационного датасета COCO
Класс наследутся от универсльного `Dataset` и реализует кастомную функцию чтения данных.

In [6]:
class COCO_Dataset(Dataset):

    def __init__(self, sublist):
        # Формируем путь к аннотациям COCO (train2017 или val2017)
        ann_file_fpath = os.path.join(COCO_ROOT, 'annotations', 'instances_' + sublist + '2017.json')

        # Загружаем аннотации COCO с помощью класса COCO
        self.coco = COCO(ann_file_fpath)

        # Получаем id категорий с именем 'person' (человека)
        self.cat_ids = self.coco.getCatIds(catNms=['person'])

        # Получаем список id изображений, содержащих объекты категории 'person'
        self.img_list = self.coco.getImgIds(catIds=self.cat_ids)

    def read_images(self, img_id):
        # img_id приходит в виде тензора, конвертируем в число Python
        img_id = int(img_id.numpy())

        # Загружаем метаданные изображения по id (список с одним элементом, берем первый)
        img_data = self.coco.loadImgs(img_id)[0]

        # Получаем относительный путь к изображению из URL (берем последние два сегмента пути)
        img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])

        # Загружаем изображение с диска, путь формируем относительно COCO_ROOT
        img = io.imread(os.path.join(COCO_ROOT, img_fname))

        # Если изображение в градациях серого (2D массив), преобразуем в RGB, продублировав каналы
        if len(img.shape) == 2:
            img = np.tile(img[..., None], (1, 1, 3))

        # Получаем id аннотаций (объектов) для данного изображения и категории 'person'
        ann_ids = self.coco.getAnnIds(imgIds=img_data['id'], catIds=self.cat_ids, iscrowd=None)

        # Загружаем сами аннотации (списки сегментаций и масок)
        anns = self.coco.loadAnns(ann_ids)

        # Создаем пустую маску (2D, размер совпадает с изображением), все нули — фон
        mask_class = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)

        # Для каждой аннотации добавляем маску объекта к общей маске (объекты суммируются)
        for i in range(len(anns)):
            mask_class += self.coco.annToMask(anns[i])

        # Преобразуем маску к бинарному виду: все пиксели с объектами — 1, остальные — 0
        mask_class = (mask_class > 0).astype(np.uint8)

        # Объединяем RGB изображение и маску по последнему измерению (каналы)
        # Теперь массив имеет 4 канала: 3 для цвета + 1 для маски
        img_combined = np.concatenate([img, mask_class[..., None]], axis=2)

        # Возвращаем объединенный массив
        return img_combined


In [7]:
COCO_dataset_train = COCO_Dataset('train')  # Создаем объект датасета для обучающей выборки COCO (train2017)
COCO_dataset_val = COCO_Dataset('val')      # Создаем объект датасета для валидационной выборки COCO (val2017)


loading annotations into memory...
Done (t=17.32s)
creating index...
index created!
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!


In [8]:
# train_ds = COCO_dataset_train.train_dataset(...)
# val_ds = COCO_dataset_val.val_dataset(...)